### Creates all form combinations

In [5]:
unusedYears = ['2000','2001','2002','2003','2004','2005','2006','2007']
year = ['2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']
month = ['January','February','March','April','May','June','July',
         'August','September','October','November','December']
states = ['Andaman and Nicobar','Andhra Pradesh','Arunachal Pradesh','Assam','Bihar','Chandigarh',
          'Chattisgarh','Dadra and Nagar Haveli','Daman and Diu','Goa','Gujarat','Haryana','Himachal Pradesh',
          'Jammu and Kashmir','Jharkhand','Karnataka','Kerala','Lakshadweep','Madhya Pradesh','Maharashtra',
          'Manipur','Meghalaya','Mizoram','Nagaland','NCT of Delhi','Odisha','Pondicherry','Punjab','Rajasthan',
          'Tamil Nadu','Telangana','Tripura','Uttar Pradesh','Uttrakhand','West Bengal']
crops = ['Brinjal','Tomato','Mango','Cauliflower','Pointed gourd (Parval)','Green Chilli']
datesAndCrops = []

for l in range(len(crops)):
    for i in range(len(year)):
        for j in range(len(month)):
            for k in range(len(states)):
                datesAndCrops.append(year[i]+' '+month[j]+' '+states[k]+' '+crops[l])
                #saveFile = month[j][:3]+year[i][2:]+states[k][:3]+crops[l][:4]+'.csv'
                #print(saveFile)
        
len(datesAndCrops)

420

### Individual query scrape automation

In [13]:
import selenium
import time
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

def checkTableExists(table):
    try:
        driver.find_element_by_id("cphBody_gridRecords")
    except NoSuchElementException:
        return False
    return True

url = 'http://agmarknet.gov.in/PriceAndArrivals/DatewiseCommodityReport.aspx'

driver = webdriver.Chrome()
driver.get(url)

year = driver.find_element_by_id("cphBody_cboYear")
year.send_keys("2008")

month = driver.find_element_by_id("cphBody_cboMonth")
month.send_keys("January")

#wait stage
wait = WebDriverWait(driver, 60)
wait.until(EC.presence_of_element_located((By.ID, 'cphBody_cboState')))

state = driver.find_element_by_id("cphBody_cboState")
state.send_keys("Bihar")

wait = WebDriverWait(driver, 60)
wait.until(EC.presence_of_element_located((By.ID, 'cphBody_cboCommodity')))

crop = driver.find_element_by_id("cphBody_cboCommodity")
crop.send_keys("Cauliflower")

wait = WebDriverWait(driver, 60)
wait.until(EC.presence_of_element_located((By.ID, 'cphBody_btnSubmit')))
print(EC.presence_of_element_located((By.ID, 'cphBody_btnSubmit')))

driver.find_element_by_name("ctl00$cphBody$btnSubmit").click()
#table
time.sleep(5)

#wait = WebDriverWait(driver, 3)
#elem = wait.until(EC.visibility_of_any_elements_located((By.ID, "cphBody_gridRecords")))


if(checkTableExists('cphBody_gridRecords')):

    tab_data = driver.find_element_by_id("cphBody_gridRecords")

    list_rows = [[cell.text for cell in row.find_elements_by_tag_name('td')]
                 for row in tab_data.find_elements_by_tag_name('tr')]
    #print(list_rows)
    df = pd.DataFrame(list_rows)
    df.columns=['Market', 'Arrival Date','Arrivals (Tonnes)','Variety','Minimum Price(Rs./Quintal)',
                'Maximum Price(Rs./Quintal)','Modal Price(Rs./Quintal)']
    df = df.iloc[1:]
    df = df.apply(lambda x: x.str.strip()).replace('', np.nan)

    for val in range(len(df['Market'])):
        if(pd.isnull(df['Market'].values[val])):
            #Sets nan value to current market
            df['Market'].values[val] = market
        else:
            #Sets the market to the current value, indicates a new market
            market=df['Market'].values[val]

    df.to_csv("Final Data/testFile.csv",index=False)
    print('File testFile completed and saved')

else:
    print('No Table')
    driver.back()
        
#print(df.at[1,6])
#should print 1250
#driver.back()

#scrape
#click back button

No Table


In [ ]:
### Cumulative query scrape automation

In [ ]:
#necessary imports
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import numpy as np
import time
import datetime
from datetime import timedelta
import os

#checks to see if element exists and returns appropriate boolean
def checkElementExistsByID(table):
    try:
        driver.find_element_by_id("cphBody_gridRecords")
    except NoSuchElementException:
        return False
    return True

#checks to see if searching term is present in the dropdown values and returns proper boolean
def checkElementInDropdown(param, val):
    inputParam = driver.find_element_by_id(param)
    if(val in inputParam.text):
        return True
    else:
        return False
#returns the average of all month timestamps for predicting runtime of next loop    
def avg(runTimes):
    if(len(runTimes)==0):
        average = 0
    else:
        average = sum(runTimes)/len(runTimes)
    return average

#writes the logs to a text file for each year and crop
def log(logs, logFileSave):
    #sets folder for log storage
    dirForLogs = 'Logs'
    #creates the folder at the correct path
    createFolder('./'+dirForLogs)
    #navigates to new directory
    standardPath = os.getcwd()+'/'+dirForLogs
    #path plus the name of file to be saved
    filename = standardPath+'/'+logFileSave

    #check if the file exists
    if os.path.exists(filename):
        appendWrite = 'a' # append if already exists
        print('Appended to existing log file')
    else:
        appendWrite = 'w' # make a new file if not
        print('Created log file')

    #open and write to file
    file = open(filename, mode= appendWrite)
    #file.write(str(logs))
    for line in logs:
        file.write(line+'\n')
    file.write('\n')
    #close the file to prevent errors
    file.close()
    
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
            print('Created directory: '+directory[2:-1])
    except OSError:
        print ('Error: Creating directory. ' +  directory)


#lists of scraped and unscraped pages after algorithm is run
scrapedPages = []
unscrapedPages = []

#sets parameters
unusedYears = ['2000','2001','2002','2003','2004','2005','2006','2007', '2008','2009','2010',
               '2011','2012','2013','2014','2016','2017','2018']
years = ['2013']
months = ['January','February','March','April']#,'May','June','July',
         #'August','September','October','November','December']
states = ['Andamans and Nicobar','Andhra Pradesh','Arunachal Pradesh','Assam','Bihar','Chandigarh',
          'Chattisgarh','Dadra and Nagar Haveli','Daman and Diu','Goa','Gujarat','Haryana','Himachal Pradesh',
          'Jammu and Kashmir','Jharkhand','Karnataka','Kerala','Lakshadweep','Madhya Pradesh','Maharashtra',
          'Manipur','Meghalaya','Mizoram','Nagaland','NCT of Delhi','Odisha','Pondicherry','Punjab','Rajasthan',
          'Tamil Nadu','Telangana','Tripura','Uttar Pradesh','Uttrakhand','West Bengal']
crops = ['Green Chilli']

#list of all pages queried
datesAndCrops = []
runTimes = []

#file names logs will be stored under
logFileSave = []
#logs from script
logs = []

url = 'http://agmarknet.gov.in/PriceAndArrivals/DatewiseCommodityReport.aspx'

#creates a webdriver using chrome and navigates to URL
driver = webdriver.Chrome()
driver.get(url)

#loops through all possible combinations of form inputs given parameters above
for l in range(len(crops)):
    for i in range(len(years)):
        #creates year crop folder for data to be stored
        createFolder('./'+years[i]+' '+crops[l]+'/')
        
        #creates the save file for logs
        logFileSave.append(years[i]+' '+crops[l]+' '+'Logs.txt')
        
        #start duration for month 
        start = time.time()
        
        #set initial time stamp
        startTS = datetime.datetime.now()
        
        for j in range(len(months)):
            for k in range(len(states)):

                datesAndCrops.append(years[i]+' '+months[j]+' '+states[k]+' '+crops[l])
                
                #creates file names for each queried page regardless if it is used
                saveFile = months[j][:3]+years[i][2:]+states[k][:4]+crops[l][:4]+'.csv'
                
                #inputs the year into the form
                inputYear = driver.find_element_by_id("cphBody_cboYear")
                inputYear.send_keys(str(years[i]))
                
                #inputs the month into the form
                inputMonth = driver.find_element_by_id("cphBody_cboMonth")
                inputMonth.send_keys(str(months[j]))

                #wait stage for state dropdown to appear
                wait = WebDriverWait(driver, 180)
                wait.until(EC.presence_of_element_located((By.ID, 'cphBody_cboState')))

                #finds dropdown for state and inputs valid state
                inputState = driver.find_element_by_id("cphBody_cboState")
                if(checkElementInDropdown('cphBody_cboState', str(states[k]))):
                    inputState.send_keys(str(states[k]))
                    
                    #wait stage for commodity dropdown to appear
                    wait.until(EC.presence_of_element_located((By.ID, 'cphBody_cboCommodity')))
                    
                    #finds dropdown for commodity and inputs valid commodity
                    inputCrop = driver.find_element_by_id('cphBody_cboCommodity')
                    if(checkElementInDropdown('cphBody_cboCommodity',str(crops[l]))):
                        inputCrop.send_keys(str(crops[l]))
                        
                        #wait stage for submit button
                        wait.until(EC.presence_of_element_located((By.ID, 'cphBody_btnSubmit')))
                        
                        #clicks submit button
                        driver.find_element_by_name("ctl00$cphBody$btnSubmit").click()
                        
                        #wait stage for page to finish loading, not necessary but there as a precaution
                        time.sleep(3)
                        
                        #checks to see if table is present on page refresh
                        if(checkElementExistsByID('cphBody_gridRecords')):
                            tab_data = driver.find_element_by_id("cphBody_gridRecords")
                            
                            #scrapes the rows from teh table
                            list_rows = [[cell.text for cell in row.find_elements_by_tag_name('td')]
                                         for row in tab_data.find_elements_by_tag_name('tr')]
                            
                            #puts rows into a dataframe
                            df = pd.DataFrame(list_rows)
                            
                            #sets the column names as they appear in the table
                            df.columns=['Market', 'Arrival Date','Arrivals (Tonnes)','Variety','Minimum Price(Rs./Quintal)',
                                        'Maximum Price(Rs./Quintal)','Modal Price(Rs./Quintal)']
                            df = df.iloc[1:]
                            
                            #replaces white space in table with nan value
                            df = df.apply(lambda x: x.str.strip()).replace('', np.nan)

                            #nan value is then converted into the appropriate market name
                            for val in range(len(df['Market'])):
                                if(pd.isnull(df['Market'].values[val])):
                                    #Sets nan value to current market
                                    df['Market'].values[val] = market
                                else:
                                    #Sets the market to the current value, indicates a new market
                                    market=df['Market'].values[val]

                            #scraped and cleaned data is saved to csv file in directory created above 
                            df.to_csv(years[i]+' '+crops[l]+"/"+saveFile,index=False)
                            logs.append('File '+saveFile+' scraped and saved')
                            print('File '+saveFile+' scraped and saved')
                            
                            #appends queried page information to scraped pages list
                            scrapedPages.append(years[i]+' '+months[j]+' '+states[k]+' '+crops[l])

                            #navigates the browser back to the form page to continue loop
                            driver.back()
                            
                        else:
                            #no table is present (no data was reported)
                            logs.append('No table for: '+saveFile+'. No file saved')
                            print('No table for: '+saveFile+'. No file saved')
                            
                            #appends queried page information to unscraped pages list
                            unscrapedPages.append(years[i]+' '+months[j]+' '+states[k]+' '+crops[l])
                            driver.back()
                        
                    else:
                        #the queried crop is not in the dropdown tab
                        logs.append('No crop for: '+saveFile+'. No file saved')
                        print('No crop for: '+saveFile+'. No file saved')
                        
                        #appends queried page information to unscraped pages list
                        unscrapedPages.append(years[i]+' '+months[j]+' '+states[k]+' '+crops[l])
                        
                        #refreshes the page to continue the loop
                        driver.refresh()
                else:
                    #the queried state is not in the dropdown tab
                    logs.append('No state for: '+saveFile+'. No file saved')
                    print('No state for: '+saveFile+'. No file saved')
                    
                    #appends queried page information to unscraped pages list
                    unscrapedPages.append(years[i]+' '+months[j]+' '+states[k]+' '+crops[l])
                    
                    #refreshes the page to continue the loop
                    driver.refresh()
                    
            #end duration for month
            end = time.time()
            #set final time stamp
            endTS = datetime.datetime.now()
            #difference between the start and end times calculated to be a positive float
            diff = (start-end)*-1

            #sets the interval for output
            interval = 'seconds'
            if(diff > 60):
                diff = diff/60
                interval = 'minutes'
            else:
                interval = 'seconds'

            #appends the difference to runTimes list
            runTimes.append(diff)
            #sets duration to the average of values in runTimes list. updates with every iteration to increase accuracy
            duration = avg(runTimes)

            #output statement for current month iteration for all states
            logs.append('It took '+str(diff)+' '+interval+' to execute '+months[j]+' '+years[i]+" for all states. Start: "+str(startTS)[11:19]+' End: '+str(endTS)[11:19])
            print('It took '+str(diff)+' '+interval+' to execute '+months[j]+' '+years[i]+" for all states. Start: "+str(startTS)[11:19]+' End: '+str(endTS)[11:19])
            #adds time to the end time stamp to predict when next month will finish executing 
            if(interval=='minutes'):
                logs.append('Around '+str(endTS+timedelta(minutes=duration))[11:19]+' is when the next month is expected to finish')
                print('Around '+str(endTS+timedelta(minutes=duration))[11:19]+' is when the next month is expected to finish')
            else:
                logs.append('Around '+str(endTS+timedelta(seconds=duration))[11:19]+' is when the next month is expected to finish')
                logs.append('\n')
                print('Around '+str(endTS+timedelta(seconds=duration))[11:19]+' is when the next month is expected to finish')
            print()

            #sets the start duration to the previous end duration
            start = end
            #sets the initial time stamp to the final timestamp
            startTS = endTS
        
            #calls method that writes logs to text file
            log(logs,logFileSave[i])
            #reset logs so data doesnt accumulate as the years increase
            logs = []
            print('Logs saved')
        
#creates a log file and saves the scraped pages list                   
with open("Scraped Page Logs.txt", "w") as output:
    output.write(str(scrapedPages))
    
#creates a log file and saves the unscraped pages list      
with open("Unscraped Page Logs.txt", "w") as output:
    output.write(str(unscrapedPages))
    
#indicates that the program has completed
driver.close()
print('Done')

totalPages = len(scrapedPages)+len(unscrapedPages)
print(str(len(scrapedPages))+' out of '+str(totalPages)+' were able to be scraped')


Created directory: 2013 Green Chilli
No state for: Jan13AndaGree.csv. No file saved
File Jan13AndhGree.csv scraped and saved
No state for: Jan13ArunGree.csv. No file saved
File Jan13AssaGree.csv scraped and saved
No table for: Jan13BihaGree.csv. No file saved
File Jan13ChanGree.csv scraped and saved
No table for: Jan13ChatGree.csv. No file saved
No state for: Jan13DadrGree.csv. No file saved
No state for: Jan13DamaGree.csv. No file saved
File Jan13GoaGree.csv scraped and saved
File Jan13GujaGree.csv scraped and saved
File Jan13HaryGree.csv scraped and saved
File Jan13HimaGree.csv scraped and saved
File Jan13JammGree.csv scraped and saved
File Jan13JharGree.csv scraped and saved
File Jan13KarnGree.csv scraped and saved
File Jan13KeraGree.csv scraped and saved
No state for: Jan13LaksGree.csv. No file saved
File Jan13MadhGree.csv scraped and saved
File Jan13MahaGree.csv scraped and saved
No state for: Jan13ManiGree.csv. No file saved
File Jan13MeghGree.csv scraped and saved
No state for: 